In [1]:
#importing Libraries
import numpy as np
import pandas as pd
from collections import Counter
import string
import re
from sklearn import linear_model
from sklearn import metrics
#from sklearn.cross_validation import train_test_split
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
#en_nlp = spacy.load('en')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
#import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import gensim
from gensim.models import Word2Vec

# Loading Data

In [2]:
df=pd.read_csv('Uttarakhand.csv')
df.head()

,tweet
0,Horrible news out of #Uttarakhand
1,Prayer for #Uttarakhand 🙏🙏
2,Ohhh God Please Save #Uttarakhand
3,Extremely Sad To Hear About The #Uttarakhand G...
4,Prayers for #Uttarakhand 🙏🙏


In [3]:
#Importing LIBRARIES
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []


# Preprocessing

The below funstion performs text cleaning(removing all unnecessary symbols & replacing the right ones)

In [4]:
#TEXT CLEANING
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].str.replace(r"can't", "can not")
    df[text_field] = df[text_field].str.replace(r"what's", "what is")
    df[text_field] = df[text_field].str.replace(r"&", "and")
    #df[text_field] = df[text_field].str.replace(r"r", "are")
    df[text_field] = df[text_field].str.replace(r"PMO", "Prime Minister Office")
    df[text_field] = df[text_field].str.replace(r"tim", "time")
    df[text_field] = df[text_field].str.replace(r"di", "difficult")
    df[text_field] = df[text_field].str.replace(r"d!ed", "died")
    df[text_field] = df[text_field].str.replace(r"m!ssing", "missing")
    df[text_field] = df[text_field].str.replace(r'[^A-Za-z0-9 ]+', ' ')
    return df

The below cell calls the function "standardize_text" to perform the cleaning

In [5]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

cleaned_text = standardize_text(df, "tweet")

In [6]:
#PRINTING THE CLEANED TEXT
cleaned_text

,tweet
0,horrible news out of uttarakhand
1,prayer for uttarakhand
2,ohhh god please save uttarakhand
3,extremely sad to hear about the uttarakhand g...
4,prayers for uttarakhand
...,...
8141,hope everyone is fine prayers uttarakhand h...
8142,what a responsible pm would do in this situati...
8143,scary scenes coming from uttarakhand praying...
8144,itbp jawans in search and rescue operation in ...


The below cell performs lemmatisation on cleaned text. Lemmatisation has been chosen because it works better than stemming.
And finally it gets converted into a list

In [7]:
messages=cleaned_text['tweet']
for i in range(0, len(messages)):
    review = messages[i]
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [8]:
clean_corpus=[]
for sen in corpus:
    sen=sen.split()
    clean_corpus.append(sen)
    

In [9]:
#PRINTING THE CLEAN TOKENISED CORPUS
print(clean_corpus)

[['horrible', 'news', 'uttarakhand'], ['prayer', 'uttarakhand'], ['ohhh', 'god', 'please', 'save', 'uttarakhand'], ['extremely', 'sad', 'hear', 'uttarakhand', 'glacier', 'break', 'praying', 'safety', 'missing', 'people', 'helpline', 'http', 'co', 'a8hxq6y3hb'], ['prayer', 'uttarakhand'], ['prayer', 'uttarakhand'], ['alert', 'uttarakhand', 'emergency', 'number', '1070', '9557444486', 'please', 'follow', 'official', 'advisory', 'uttarakhand', 'chamoli'], ['heavy', 'devastation', 'due', 'glacier', 'breakage', 'chamoli', 'uttrakhand', 'sad', 'news', 'geographical', 'issue', 'country', 'http', 'co', 'hfmkb6d9bx'], ['wish', 'everyone', 'stay', 'safe', 'uttarakhand', 'tragedy', 'uttarakhand'], ['lord', 'shiva', 'please', 'save', 'uttarakhand'], ['difficultstressing', 'hear', 'glacier', 'breaking', 'uttarakhand', 'praying', 'everyone', 'safety'], ['uttarakhand', 'tragic', 'event', 'unfoldifficultng', 'http', 'co', 'bqio6lfswg'], ['praying', 'safety', 'uttarakhand'], ['happening', 'scary', 'pra

# Converting text to vectors

The below cell makes use of word2vec model to obtain word embeddings.

In [10]:
model = gensim.models.Word2Vec(
        clean_corpus,
        size=100,
        window=10,
        min_count=2,
        workers=10,
        iter=10)

In [11]:
#SAVING THE MODEL
model_train=model.wv.save_word2vec_format('train.model')

In [12]:
#LOADING THE QUESTION AND ANSWER CSV FILE
qa_data=pd.read_csv('qa_data.csv')
qa_data.head()

,question,answer
0,What is the helpline number?,1070 9557444486
1,What happened in Uttarakhand?,Heavy Devastation Due To Glacier breakage in C...
2,How many people are affected?,A large population is affected
3,What is the helpline website?,https://t.co/L7i51JaKS9
4,Where has this happened in uttarakhand,Chamoli


In [13]:
#PERFORMING THE CLEANING ON QUESTION AND ANSWER DATASET
cleaned_text_q = standardize_text(qa_data, "question")
cleaned_text_a = standardize_text(qa_data, "answer")

In [14]:
cleaned_text_q

,question,answer
0,what is the helpline number,1070 9557444486
1,what happened in uttarakhand,heavy devastation due to glacier breakage in c...
2,how many people are affected,a large population is affected
3,what is the helpline website,https t co l7i51jaks9
4,where has this happened in uttarakhand,chamoli
5,are there any deaths reported,yes but there is no exact number


In [15]:
cleaned_text_a

,question,answer
0,what is the helpline number,1070 9557444486
1,what happened in uttarakhand,heavy devastation due to glacier breakage in c...
2,how many people are affected,a large population is affected
3,what is the helpline website,https t co l7i51jaks9
4,where has this happened in uttarakhand,chamoli
5,are there any deaths reported,yes but there is no exact number


In [16]:
#PERFORMING LEMMATISATION ON QUESTIONS AND SAVING IN LIST
q_corpus=[]
questions=cleaned_text_q['question']
for i in range(0, len(questions)):
    review = questions[i]
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    q_corpus.append(review)

In [17]:
q_corpus

['helpline number',
 'happened uttarakhand',
 'many people affected',
 'helpline website',
 'happened uttarakhand',
 'death reported']

In [18]:
#PERFORMING LEMMATISATION ON ANSWERS AND SAVING IN LIST
a_corpus=[]
answers=cleaned_text_a['answer']
for i in range(0, len(answers)):
    review = answers[i]
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    a_corpus.append(review)

In [19]:
a_corpus

['1070 9557444486',
 'heavy devastation due glacier breakage chamoli uttrakhand sad news geographical issue country',
 'large population affected',
 'http co l7i51jaks9',
 'chamoli',
 'yes exact number']

In [20]:
clean_corpus_q=[]
for sen in q_corpus:
    sen=sen.split()
    clean_corpus_q.append(sen)
    

In [21]:
clean_corpus_q

[['helpline', 'number'],
 ['happened', 'uttarakhand'],
 ['many', 'people', 'affected'],
 ['helpline', 'website'],
 ['happened', 'uttarakhand'],
 ['death', 'reported']]

In [22]:
clean_corpus_a=[]
for sen in a_corpus:
    sen=sen.split()
    clean_corpus_a.append(sen)
    

In [23]:
clean_corpus_a

[['1070', '9557444486'],
 ['heavy',
  'devastation',
  'due',
  'glacier',
  'breakage',
  'chamoli',
  'uttrakhand',
  'sad',
  'news',
  'geographical',
  'issue',
  'country'],
 ['large', 'population', 'affected'],
 ['http', 'co', 'l7i51jaks9'],
 ['chamoli'],
 ['yes', 'exact', 'number']]

In [24]:
#CONVERTING QUESTIONS INTO WORD EMBEDDINGS
model_q = gensim.models.Word2Vec(
        clean_corpus_q,
        size=100,
        window=10,
        min_count=2,
        workers=10,
        iter=10)

In [25]:
#CONVERTING ANSWERS INTO WORD EMBEDDINGS
model_a = gensim.models.Word2Vec(
        clean_corpus_a,
        size=100,
        window=10,
        min_count=2,
        workers=10,
        iter=10)

In [ ]:

#CHECKING THE SIMILARITY BETWEEN SENTENCES
model_res = model.wv.load_word2vec_format('train.model')  

new_sentence = "Heavy Devastation Due To Glacier breakage in Chamoli, Uttrakhand.Very sad news...Geographical issues of our country".split(" ")  
model_res.most_similar(positive=[model.infer_vector(new_sentence)],topn=5)